## 1. Set up the scraper
* Download data from nfl.com
* Use internal api with Bearer Key

### 1.1 Get a working example for Bearer token

In [2]:
import time
from browsermobproxy import Server
from selenium import webdriver
import json
import os
BMP_PATH = os.path.abspath('browsermob-proxy-2.1.4/bin/browsermob-proxy.bat')

# Start BrowserMob Proxy
server = Server(BMP_PATH)
server.start()
proxy = server.create_proxy()

# Configure Chrome with the proxy
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument(f'--proxy-server={proxy.proxy}')
chrome_options.add_argument('--ignore-certificate-errors')
chrome_options.add_argument('--headless')
driver = webdriver.Chrome(options=chrome_options)
# Navigate to the website
driver.get("https://nfl.com/scores")

# Start capturing network traffic
proxy.new_har("nfl", options={'captureHeaders': True})

# Wait for the header with the name "Authorization"
timeout = 60  # Adjust the timeout as needed
def get_auth_token():
    start_time = time.time()
    while time.time() - start_time < timeout:
        har = proxy.har
        for entry in har['log']['entries']:
            request_headers = entry['request']['headers']
            for header in request_headers:
                if header['name'] == 'Authorization':
                    return header['value']
        time.sleep(1)
    return None
auth_token = get_auth_token()

# Store the auth token
auth_path = 'auth.json'
auth_json = {
    'time': time.time(),
    'token': auth_token
}
with open(auth_path, 'w') as f:
    json.dump(auth_json, f)

# Stop capturing network traffic
har = proxy.har

# Close the browser and BrowserMob Proxy
driver.quit()
server.stop()

### 1.2 Create a class that manages requests to nfl.com

In [5]:
from nfl_client import NFLClient
client = NFLClient(headless=True)

In [7]:
client.load_auth_token(store_har=True)
print('Auth Token:', client.auth_token)

Auth Token: Bearer eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJjbGllbnRJZCI6ImU1MzVjN2MwLTgxN2YtNDc3Ni04OTkwLTU2NTU2ZjhiMTkyOCIsImNsaWVudEtleSI6IjRjRlVXNkRtd0pwelQ5TDdMckczcVJBY0FCRzVzMDRnIiwiaXNzIjoiTkZMIiwiZGV2aWNlSWQiOiI0YTIzZGM2My00YmQ2LTQ0OTMtYWVmNC0yNDk0OGRkNGMxYTUiLCJwbGFucyI6W3sicGxhbiI6ImZyZWUiLCJleHBpcmF0aW9uRGF0ZSI6IjIwMjQtMTEtMDQiLCJzb3VyY2UiOiJORkwiLCJzdGFydERhdGUiOiIyMDIzLTExLTA1Iiwic3RhdHVzIjoiQUNUSVZFIiwidHJpYWwiOmZhbHNlfV0sIkRpc3BsYXlOYW1lIjoiV0VCX0RFU0tUT1BfREVTS1RPUCIsIk5vdGVzIjoiIiwiZm9ybUZhY3RvciI6IkRFU0tUT1AiLCJsdXJhQXBwS2V5IjoiU1pzNTdkQkdSeGJMNzI4bFZwN0RZUSIsInBsYXRmb3JtIjoiREVTS1RPUCIsInByb2R1Y3ROYW1lIjoiV0VCIiwiY2l0eSI6ImFwcGxlIHZhbGxleSIsImNvdW50cnlDb2RlIjoiVVMiLCJkbWFDb2RlIjoiODAzIiwiaG1hVGVhbXMiOlsiMTA0MDI1MTAtODkzMS0wZDVmLTk4MTUtNzliYjc5NjQ5YTY1IiwiMTA0MDI1MjAtOTZiZi1lOWYyLTRmNjgtODUyMWNhODk2MDYwIiwiMTA0MDQ0MDAtM2IzNS0wNzNmLTE5N2UtMTk0YmI4MjQwNzIzIl0sInJlZ2lvbiI6IkNBIiwiYnJvd3NlciI6IkNocm9tZSBIZWFkbGVzcyIsImNlbGx1bGFyIjpmYWxzZSwiZW52aXJvbm1lbnQiOiJwcm9kdWN0aW9u

### 1.3 Convert har entry into json
1. Url to json
2. Convert har to json 

In [1]:
from nfl_client.url_utils import urlparse_json
url = '/football/v2/games/season/2023/seasonType/REG/week/9'
urlparse_json(url)

{'/football/v2/games/season/{season}/seasonType/REG/week/{week}': {'query_parameters': {},
  'path_parameters': {'season': {'type': 'integer', 'example': 2023},
   'week': {'type': 'integer', 'example': 9}}}}

In [ ]:
from nfl_client.url_utils import har_entry_parse
import json
with open('nfl_client_data/sample_har_entry.json', 'r') as f:
    data = json.load(f)

url_data = har_entry_parse(data)
url_data

#### 1.3.1 Create something to convert this into openapi

In [6]:
from nfl_client.url_utils import generate_openapi_spec

# Example
print(url_data)
openapi_yaml = generate_openapi_spec(url_data)
with open('openapi.yaml', 'w') as f:
    f.write(openapi_yaml)

{'/football/v2/games/season/{season}/blah/{blah_id}': {'query_parameters': {'withExternalIds': {'type': 'boolean', 'example': 'true'}}, 'path_parameters': {'blah_id': {'type': 'string', 'example': 'b07eaa67-f053-11ed-b4a7-bab79e4492fa'}}, 'response_sample': {'id': 'b07eaa67-f053-11ed-b4a7-bab79e4492fa', 'homeTeam': {'id': '10402310-a47e-10ea-7442-16b633633637', 'currentLogo': 'https://static.www.nfl.com/{formatInstructions}/league/api/clubs/logos/KC', 'fullName': 'Kansas City Chiefs'}, 'awayTeam': {'id': '10402700-1662-d8ad-f45c-0b0ea460d045', 'currentLogo': 'https://static.www.nfl.com/{formatInstructions}/league/api/clubs/logos/MIA', 'fullName': 'Miami Dolphins'}, 'category': None, 'date': '2023-11-05', 'time': '2023-11-05T14:30:00Z', 'broadcastInfo': {'homeNetworkChannels': ['NFL NETWORK'], 'awayNetworkChannels': ['NFL NETWORK'], 'internationalWatchOptions': [{'countryCode': 'CA', 'broadcasters': ['DAZN Canada', 'RDS', 'TSN']}, {'countryCode': 'GB', 'broadcasters': ['Sky Sports', 'NF

#### 1.3.2 Put it all together

In [1]:
from nfl_client import NFLClient
client = NFLClient()
url_json = client.download_endpoints()
len(url_json)

8

## 2. Collect data into tables from the endpoints

* /football/v2/games/season/{season}/seasonType/{seasonType}/week/{week}
* /football/v2/standings
* /football/v2/stats/live/game-summaries/{game-summary_id}

In [4]:
from nfl_client import NFLClient
client = NFLClient()
r = client.get_teams(2023)
for team in r.json()['teams']:
    print(team['fullName'])

https://api.nfl.com/football/v2/teams/history
AFC Pro Bowl Team
Arizona Cardinals
Atlanta Falcons
Baltimore Ravens
Buffalo Bills
Carolina Panthers
Chicago Bears
Cincinnati Bengals
Cleveland Browns
Dallas Cowboys
Denver Broncos
Detroit Lions
Green Bay Packers
Houston Texans
Indianapolis Colts
Jacksonville Jaguars
Kansas City Chiefs
Las Vegas Raiders
Los Angeles Chargers
Los Angeles Rams
Miami Dolphins
Minnesota Vikings
New England Patriots
New Orleans Saints
New York Giants
New York Jets
NFC Pro Bowl Team
Philadelphia Eagles
Pittsburgh Steelers
San Francisco 49ers
Seattle Seahawks
Tampa Bay Buccaneers
Tennessee Titans
Washington Commanders
